In [1]:
#!/usr/bin/env python
# coding: utf-8
%load_ext autoreload
%autoreload 2
#------ Load necessary packages ------#
import sys
import os
import numpy as np
# import pandas as pd
#from common_utils import compute_metrics
sys.path.append("../")

# load datasets
from aif360.datasets import AdultDataset, GermanDataset, BankDataset, CompasDataset, BinaryLabelDataset, CelebADataset

# load metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
# from aif360.metrics.utils import compute_boolean_conditioning_vector

# load preprocessing algorithm
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas
# from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
# from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_german
# from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
# from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult

# load algorithms
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing.prejudice_remover import PrejudiceRemover
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
import keras
# load other packages
import tensorflow as tf
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale, StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score
from function_1 import *
from fair_main_update1 import *
#import fair_bak
import fair_ablation
import easydict 
import argparse
from plot import plot

import pandas as pd
# from sklearn.metrics import classification_report
# from IPython.display import Markdown, display

/home/taeuk/anaconda3/envs/aif/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/taeuk/anaconda3/envs/aif/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/taeuk/anaconda3/envs/aif/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/taeuk/anaconda3/envs/aif/lib/python3.6/site-packages

In [2]:
args = easydict.EasyDict({ "dataset": 'celebA', "senstive_feature": 0, "batch_size": 64, "repeat": 1, "gpu_id": '3', "epochs": 5000, "network" : 'vgg16' })

dataset_used = args.dataset
protected_attribute_used = args.senstive_feature
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

if dataset_used == "adult":
    dataset_orig = AdultDataset()
    # dataset_orig = load_preproc_data_adult()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        sens_attr = 'sex'
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        sens_attr = 'race'

elif dataset_used == "german":
    dataset_orig = GermanDataset()
    # dataset_orig.labels = dataset_orig.labels-1
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        sens_attr = 'sex'
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        sens_attr = 'age'

elif dataset_used == "compas":
    dataset_orig = CompasDataset()
    # dataset_orig = load_preproc_data_compas()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        sens_attr = 'sex'
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        sens_attr = 'race'

elif dataset_used == "bank":
    dataset_orig = BankDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        sens_attr = 'age'
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        sens_attr = 'age'
        
elif dataset_used == 'celebA':
    dataset_orig = CelebADataset(args.network)
    privileged_groups = [{'gender': 0}]
    unprivileged_groups = [{'gender': 1}]
    sens_attr = 'gender'
    dataset_orig.features[:,0:2048] += 1e-8 # male = 2, female = 1


In [3]:
dataset_used = 'celebA'
dataset_orig = CelebADataset('vgg16')
privileged_groups = [{'gender': 0}]
unprivileged_groups = [{'gender': 1}]
sens_attr = 'gender'

In [4]:
        
sens_idx = (dataset_orig.feature_names).index(sens_attr)

min_max_scaler = MaxAbsScaler()
dataset_orig.features[:,:-1] = min_max_scaler.fit_transform(dataset_orig.features)[:,:-1]

dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

repeat = args.repeat

total_acc = np.zeros((11, repeat))
total_balanced_acc = np.zeros((11, repeat))
total_disimpact = np.zeros((11, repeat))
total_eqopp_diff = np.zeros((11, repeat))

total_aveodds_diff = np.zeros((11, repeat))
total_theil_idx = np.zeros((11, repeat))
stat_parity_diff = np.zeros((11, repeat))

total_tpr = np.zeros((11, repeat))
total_tpr_priv = np.zeros((11, repeat))
total_tpr_unpriv = np.zeros((11, repeat))

total_fpr = np.zeros((11, repeat))
total_fpr_priv = np.zeros((11, repeat))
total_fpr_unpriv = np.zeros((11, repeat))
total_fpr_diff = np.zeros((11, repeat))

total_acc_priv = np.zeros((11, repeat))
total_acc_unpriv = np.zeros((11, repeat))

In [5]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
sens_idx = (dataset_orig_train.feature_names).index(sens_attr)


tmp = dataset_orig_train.labels.astype(int)
if dataset_used == "german":
    tmp = tmp - 1
targets = tmp.reshape(-1).tolist()
y_train = np.eye(np.unique(tmp).shape[0])[targets]

tmp = dataset_orig_valid.labels.astype(int)
if dataset_used == "german":
    tmp = tmp - 1
targets = tmp.reshape(-1).tolist()
y_valid = np.eye(np.unique(tmp).shape[0])[targets]

tmp = dataset_orig_test.labels.astype(int)
if dataset_used == "german":
    tmp = tmp - 1
targets = tmp.reshape(-1).tolist()
y_test = np.eye(np.unique(tmp).shape[0])[targets]


# for lr in [1e-2, 1e-3, 1e-4, 1e-5]:
#     for lr_t in [1e-2, 1e-3, 1e-4, 1e-5]:
        
lr = 1e-2
lr_t = 1e-3
print(lr)
print(lr_t)

dataset_orig_test.features[:,-1]+=1
dataset_orig_train.features[:,-1]+=1
dataset_orig_valid.features[:,-1]+=1

outpred = fair_shapley(dataset_orig_train.features, y_train, dataset_orig_valid.features, y_valid, dataset_orig_test.features, \
                       sens_idx, sess, args.batch_size, args.epochs, lr, lr_t)

dataset_orig_test.features[:,-1]-=1
dataset_orig_train.features[:,-1]-=1
dataset_orig_valid.features[:,-1]-=1


dataset_pred = dataset_orig_test.copy()

dataset_pred.labels = np.argmax(outpred, axis=1)


classified_metric = ClassificationMetric(dataset_orig_test,
                                     dataset_pred,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)
metric_pred = BinaryLabelDatasetMetric(dataset_pred,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)

total_acc[9, 0] = classified_metric.accuracy()
total_balanced_acc[9, 0] = 0.5 * (classified_metric.true_positive_rate() +
                                     classified_metric.true_negative_rate())
total_disimpact[9, 0] = metric_pred.disparate_impact()
total_eqopp_diff[9, 0] = classified_metric.equal_opportunity_difference()
total_aveodds_diff[9, 0] = classified_metric.average_odds_difference()
total_theil_idx[9, 0] = classified_metric.theil_index()
stat_parity_diff[9, 0] = classified_metric.statistical_parity_difference()
total_tpr[9, 0] = classified_metric.recall()


#total_tpr[9, 0] = classified_metric_orig_test.recall()
total_fpr_diff[9, 0] = classified_metric.false_positive_rate_difference()
total_fpr[9, 0] = classified_metric.false_positive_rate()

total_tpr_unpriv[9, 0] = classified_metric.performance_measures(True)['TPR']
total_tpr_priv[9, 0] = classified_metric.performance_measures(False)['TPR']
total_fpr_unpriv[9, 0] = classified_metric.performance_measures(True)['FPR']
total_fpr_priv[9, 0] = classified_metric.performance_measures(False)['FPR']


total_acc_priv[9, 0] = classified_metric.performance_measures(False)['ACC']
total_acc_unpriv[9, 0] = classified_metric.performance_measures(True)['ACC']

print('FAIAS')
print('Overall TPR : {:.3f}'.format(total_tpr[9].item()))
print('Priv TPR : {:.3f}'.format(total_tpr_priv[9].item()))
print('Unpriv TPR : {:.3f}'.format(total_tpr_unpriv[9].item()))
print('Eq.Opp diff: {:.3f}'.format(total_eqopp_diff[9].item()))
print()
print('Overall FPR : {:.3f}'.format(total_fpr[9].item()))
print('Priv FPR : {:.3f}'.format(total_fpr_priv[9].item()))
print('Unpriv FPR : {:.3f}'.format(total_fpr_unpriv[9].item()))
print('FPR diff : {:.3f}'.format(total_fpr_diff[9].item()))
print()
print('Overall ACC : {:.3f}'.format(total_acc[9].item()))
print('Priv ACC : {:.3f}'.format(total_acc_priv[9].item()))
print('Unpriv ACC : {:.3f}'.format(total_acc_unpriv[9].item()))
print('ACC diff : {:.3f}'.format(abs(total_acc_unpriv[9].item() - total_acc_priv[9].item())))
print()
print()
print('Disparate Impact : {:.3f}'.format(total_disimpact[9].item()))
print('Eq.Opp diff : {:.3f}'.format(total_eqopp_diff[9].item()))
print('Average Odds Diff : {:.3f}'.format(total_aveodds_diff[9].item()))
print('Theil Index : {:.3f}'.format(total_theil_idx[9].item()))
print('Stat parity diff : {:.3f}'.format(stat_parity_diff[9].item()))
print()

0.01
0.001
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



 best model saved 

 
Epoch: 1900, tot_acc: 0.734375, pred_acc: 0.71875, diff_acc: 0.796875, pred_loss: 1.0769, diff_loss: 1.0142, acc: 0.4688, TPR female: 0.9627, TPR male: 0.8208, Eq_opp: 0.1419


 best model saved 

 
Epoch: 2700, tot_acc: 0.734375, pred_acc: 0.78125, diff_acc: 0.46875, pred_loss: 0.6549, diff_loss: 1.3092, acc: 0.5, TPR female: 0.9742, TPR male: 0.8758, Eq_opp: 0.0984
FAIAS
[0.9440553]
[0.84977487]
[0.97145236]
[-0.12167749]

[0.36198791]
[0.2214074]
[0.58644743]
[-0.36504004]

[0.7941052]
[0.79823769]
[0.79116013]
[0.00707755]


[0.46612813]
[-0.12167749]
[-0.24335877]
[0.07528065]
[-0.45220722]



In [7]:
print('FAIAS')
print('Overall TPR : {:.3f}'.format(total_tpr[9].item()))
print('Priv TPR : {:.3f}'.format(total_tpr_priv[9].item()))
print('Unpriv TPR : {:.3f}'.format(total_tpr_unpriv[9].item()))
print('Eq.Opp diff: {:.3f}'.format(total_eqopp_diff[9].item()))
print()
print('Overall FPR : {:.3f}'.format(total_fpr[9].item()))
print('Priv FPR : {:.3f}'.format(total_fpr_priv[9].item()))
print('Unpriv FPR : {:.3f}'.format(total_fpr_unpriv[9].item()))
print('FPR diff : {:.3f}'.format(total_fpr_diff[9].item()))
print()
print('Overall ACC : {:.3f}'.format(total_acc[9].item()))
print('Priv ACC : {:.3f}'.format(total_acc_priv[9].item()))
print('Unpriv ACC : {:.3f}'.format(total_acc_unpriv[9].item()))
print('ACC diff : {:.3f}'.format(abs(total_acc_unpriv[9].item() - total_acc_priv[9].item())))
print()
print()
print('Disparate Impact : {:.3f}'.format(total_disimpact[9].item()))
print('Eq.Opp diff : {:.3f}'.format(total_eqopp_diff[9].item()))
print('Average Odds Diff : {:.3f}'.format(total_aveodds_diff[9].item()))
print('Theil Index : {:.3f}'.format(total_theil_idx[9].item()))
print('Stat parity diff : {:.3f}'.format(stat_parity_diff[9].item()))
print()

FAIAS
Overall TPR : 0.944
Priv TPR : 0.850
Unpriv TPR : 0.971
Eq.Opp diff: -0.122

Overall FPR : 0.362
Priv FPR : 0.221
Unpriv FPR : 0.586
FPR diff : -0.365

Overall ACC : 0.794
Priv ACC : 0.798
Unpriv ACC : 0.791
ACC diff : 0.007


Disparate Impact : 0.466
Eq.Opp diff : -0.122
Average Odds Diff : -0.243
Theil Index : 0.075
Stat parity diff : -0.452

